In [1]:
import pandas as pd
import numpy as np

## 유동인구

2019년 1월, 2월 유동인구 데이터가 존재하지 않음<br/>
2019년 11월, 12월로 채워넣기

In [58]:
def smc_preprocessing(train_data_path, test_data_path, new_data_path, train_name, test_name):
    
    # 1. 유동인구
    df_categories = ["1903","1904","1905","1906","1907","1908","1909","1910","1911","1912","2006"]
    dfs = {}
    df_list = []

    for df in df_categories:
        df_name = df
        filename = new_data_path+'/{}.csv'.format(df)
        dfs[df_name] = pd.read_csv(filename, index_col=0).reset_index()
    
    # 1월, 2월은 11월, 12월 유동인구로 대체
    dfs["1901"] = pd.read_csv(new_data_path+'/1911.csv')
    dfs["1902"] = pd.read_csv(new_data_path+'/1912.csv')
    
    for df in dfs:
        dfs[df]["일"] = dfs[df]["일자"].apply(lambda x:int(str(x)[6:8]))
        dfs[df]["월"] = dfs[df]["일자"].apply(lambda x:int(str(x)[4:6]))
        
    dfs["1901"]["월"] = 1
    dfs["1902"]["월"] = 2
    dfs[df].drop(columns=["일자"], inplace=True)
    
    new_df = None
    for df in dfs:
        new_df = pd.concat([new_df, dfs[df]])
        
    new_df.rename(columns={"시간(1시간단위)":"시간"}, inplace=True)
    new_df2= new_df.groupby(["월","일","시간","성별","연령대(10세단위)"])["유동인구수"].sum().reset_index()
    
    all_fit = new_df2.groupby(["월","일","시간"])["유동인구수"].sum().reset_index().rename(columns={"유동인구수":"전체유동인구수"})
    mean_fit = new_df2.groupby(["월","일","시간"])["유동인구수"].mean().reset_index().rename(columns={"유동인구수":"평균유동인구수"})
    sd_fit = new_df2.groupby(["월","일","시간"])["유동인구수"].std().reset_index().rename(columns={"유동인구수":"유동인구수표준편차"})

    new_data = new_df2.pivot_table(columns=["성별","연령대(10세단위)"], values=["유동인구수"], index=["월","일","시간"]).reset_index()
    
    
    # 2. 기존 데이터 전처리
    trainpath = train_data_path + '/{}.xlsx'.format(train_name)
    testpath = test_data_path + '/{}.xlsx'.format(test_name)
    
    train = pd.read_excel(trainpath, header=1)
    test = pd.read_excel(testpath, header=1)
    
    
    
    ## 1) 취급액이 없는 경우 제거
    train = train.dropna(subset=["취급액"])
    
    ## 2) 취급액이 판매단가보다 작은 경우 제거
    train = train.loc[lambda x:x.판매단가 <= x.취급액]
    
    ## 3) train과 test 결합
    data = pd.concat([train, test], axis=0)
    
    ## 4) 방송일시 분할
    data["연"] = data["방송일시"].dt.year
    data["월"] = data["방송일시"].dt.month
    data["일"] = data["방송일시"].dt.day
    data["시간"] = data["방송일시"].dt.hour
    data["분"] = data["방송일시"].dt.minute
    data["요일"] = data["방송일시"].dt.weekday
    
    ## 5) 노출 NA imputation - 같은 방송일시의 노출 시간
    data["노출(분)"] = data["노출(분)"].fillna(method="ffill")    
    ## 6) 100원 단위에서 반올림한 판매단가
    data["판매단가_100"] = np.round(data["판매단가"], -3)
    data["판매단가_만원대"] = data["판매단가"].apply(lambda x:np.trunc(x/10000)*10000)
    
    ## 7) 방송순서
    data["일별방송순서"] = data.groupby(["연","월","일","상품명"])["상품코드"].rank(method="first")
    data["월별방송순서"] = data.groupby(["연","월","상품명"])["상품코드"].rank(method="first")
    
    ## 8) 노출 시간 반올림
    data["노출_5"] = data["노출(분)"].apply(lambda x: round(x/5)*5)
    data["노출_10"] = data["노출(분)"].apply(lambda x: round(x,-1))
    
    ## 9) 계절
    data["계절"] = data["월"].map({3:"봄",4:"봄",5:"봄",6:"여름",7:"여름",8:"여름",9:"가을",10:"가을",11:"가을",12:"겨울",1:"겨울",2:"겨울"})
    
    # 3. 외부데이터와 결합
    tmp = pd.merge(data, new_data, on=["월","일","시간"], how="left")
    tmp2 = pd.merge(tmp, all_fit, on = ["월","일","시간"], how="left")
    tmp3 = pd.merge(tmp2, mean_fit, on=["월","일","시간"], how="left")
    final = pd.merge(tmp3, sd_fit, on=["월","일","시간"], how="left")
    
    final.fillna(method="ffill", inplace=True)
    drop_list = [x for x in final.columns if x in train.columns]
    
    wanted_data = final.drop(columns = drop_list)
    
    return wanted_data